In [1]:
from qiskit_optimization.applications import Knapsack
import pandas as pd

from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler

In [2]:
data = pd.read_csv('40points.csv')

In [3]:
data

,77,95
0,44,70
1,15,85
2,67,31
3,75,10
4,91,1
5,2,7
6,46,53
7,16,35
8,68,11
9,74,91


In [4]:
values = data.iloc[:,0]
weights = data.iloc[:,1]
max_weight = int(0.7 * sum(weights))

In [5]:
values

0     44
1     15
2     67
3     75
4     91
5      2
6     46
7     16
8     68
9     74
10    43
11    14
12    64
13    73
Name: 77, dtype: int64

In [6]:
sum( weights)

593

In [7]:
# print(f'values = {values}, weights={weights},max_weight={max_weight}')


In [8]:
prob = Knapsack(values=values, weights=weights, max_weight=max_weight)
qp = prob.to_quadratic_program()
print(qp.prettyprint())

Problem name: Knapsack

Maximize
  44*x_0 + 15*x_1 + 43*x_10 + 14*x_11 + 64*x_12 + 73*x_13 + 67*x_2 + 75*x_3
  + 91*x_4 + 2*x_5 + 46*x_6 + 16*x_7 + 68*x_8 + 74*x_9

Subject to
  Linear constraints (1)
    70*x_0 + 85*x_1 + 72*x_10 + 83*x_11 + 34*x_12 + 10*x_13 + 31*x_2 + 10*x_3
    + x_4 + 7*x_5 + 53*x_6 + 35*x_7 + 11*x_8 + 91*x_9 <= 415  'c0'

  Binary variables (14)
    x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 x_10 x_11 x_12 x_13



In [9]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [10]:
# Numpy Eigensolver
meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))

objective function value: 647.0
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=1.0, x_4=1.0, x_5=1.0, x_6=1.0, x_7=0.0, x_8=1.0, x_9=1.0, x_10=1.0, x_11=0.0, x_12=1.0, x_13=1.0
status: SUCCESS

solution: [0, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13]


In [11]:
from qiskit_algorithms import QAOA


In [ ]:
# QAOA
meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, sampler=Sampler(), optimizer=COBYLA()))
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))
print("\ntime:", result.min_eigen_solver_result.optimizer_time)

In [ ]:
from qiskit_optimization.converters import QuadraticProgramToQubo


In [ ]:
# the same knapsack problem instance as in the previous section
prob = Knapsack(values=values, weights=weights, max_weight=max_weight)
qp = prob.to_quadratic_program()
print(qp.prettyprint())

In [ ]:
# intermediate QUBO form of the optimization problem
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
print(qubo.prettyprint())

In [ ]:
# qubit Hamiltonian and offset
op, offset = qubo.to_ising()
print(f"num qubits: {op.num_qubits}, offset: {offset}\n")
print(op)